In [139]:
from coppeliasim_zmqremoteapi_client import *
import math
import numpy as np
import threading
PI = math.pi
client = RemoteAPIClient()
sim = client.getObject('sim')
cancel = threading.Event()

### Get Handles and Setup Joints

In [140]:
joint_handles = []
finger_handles = []
link_handles = []
connector = sim.getObject('/ROBOTIQ_85_attachPoint')
objectSensor = sim.getObject('/ROBOTIQ_85_attachProxSensor')
attached_shape = -1
for i in range(1,7):
    joint_handle = sim.getObject('/UR5_joint'+str(i))
    joint_handles.append(joint_handle)
    sim.setJointMode(joint_handle, sim.jointmode_dynamic, 0)
    sim.setObjectInt32Param(joint_handle, sim.jointintparam_dynctrlmode, sim.jointdynctrl_velocity)
    sim.setObjectInt32Param(joint_handle, sim.jointintparam_motor_enabled, 1)
    sim.setObjectInt32Param(joint_handle, sim.jointintparam_velocity_lock, 1)
    sim.setJointTargetVelocity(joint_handle, 0)
    link_handle = sim.getObject('/UR5_link'+str(i+1)+'_visible')
    link_handles.append(link_handle)

    if i < 3:
        finger_handle = sim.getObject('/ROBOTIQ_85_active'+str(i))
        finger_handles.append(finger_handle)
        sim.setJointMode(finger_handle, sim.jointmode_dynamic, 0)
        sim.setObjectInt32Param(finger_handle, sim.jointintparam_dynctrlmode, sim.jointdynctrl_velocity)
        sim.setObjectInt32Param(finger_handle, sim.jointintparam_motor_enabled, 1)
        sim.setObjectInt32Param(finger_handle, sim.jointintparam_velocity_lock, 1)
        sim.setJointTargetVelocity(finger_handle, 0.01)


## DH Parameters (Forward Kinematics)

In [141]:
from math import sin, cos
from functools import reduce
frame0 = sim.getObject('/frame0')
frame6 = sim.getObject('/frame6')
d1 = 0.0892
a2 = 0.4251
a3 = 0.3923
d4 = 0.1092
d5 = 0.09465
d6 = 0.176
dh_table = np.array([[0, PI/2, d1, 0],
                     [a2, 0, 0, PI/2],
                     [a3, 0, 0, 0],
                     [0, -PI/2, d4, -PI/2],
                     [0, PI/2, d5, 0],
                     [0, 0, d6, 0]])


def forward_kinematics(theta):
    A = np.array([np.array([[cos(theta[i]+dh_table[i][3]), -sin(theta[i]+dh_table[i][3])*cos(dh_table[i][1]),
                             sin(theta[i]+dh_table[i][3])*sin(dh_table[i][1]), dh_table[i][0]*cos(theta[i]+dh_table[i][3])],
                            [sin(theta[i]+dh_table[i][3]), cos(theta[i]+dh_table[i][3])*cos(dh_table[i][1]),
                             -cos(theta[i]+dh_table[i][3])*sin(dh_table[i][1]), dh_table[i][0]*sin(theta[i]+dh_table[i][3])],
                            [0, sin(dh_table[i][1]), cos(dh_table[i][1]), dh_table[i][2]],
                            [0, 0, 0, 1]]) for i in range(6)])

    T = reduce(np.dot, A)
    return T

## Ground Truth

In [142]:
def get_ground_truth():
    np.array(sim.getObjectPosition(frame6, frame0))
    return np.vstack([np.array(sim.poseToMatrix(sim.getObjectPose(frame6, frame0)), dtype=np.float64).reshape(3,4), [0, 0, 0, 1]])

### Start Simulation

In [143]:
sim.startSimulation()

1

### Useful functions

In [144]:
def limit_angle(angle):
    angle_mod = angle % (4 * np.pi)
    if angle_mod > 2* np.pi:
        angle_mod -= 4 * np.pi
    return angle_mod

In [145]:
def get_positions(joint_handles):
    positions = np.zeros(len(joint_handles))
    for joint_handle in joint_handles:
        positions[joint_handles.index(joint_handle)] = limit_angle(sim.getJointPosition(joint_handle))
    return positions

### Move to Configuration

In [146]:
def task(joint_handles, target):
    global cancel
    if cancel.is_set():
        return
    kp = np.array([.4, .4, .4, .4, .4, .4])
    limit = np.array([0.4, 0.4, 0.4, 0.4, 0.4, 0.4])
    error = np.array(target) - get_positions(joint_handles)
    threshold = np.array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01])
    while np.max(np.abs(error)) > np.max(threshold):
        if cancel.is_set():
            return
        control_signal = kp*error
        mask = control_signal > limit
        control_signal[mask] = limit[mask]
        mask = control_signal < -limit
        control_signal[mask] = -limit[mask]
        for joint_handle in joint_handles:
            if np.abs(error[joint_handles.index(joint_handle)]) < threshold[joint_handles.index(joint_handle)]:
                sim.setJointTargetVelocity(joint_handle, 0)
            else:
                sim.setJointTargetVelocity(joint_handle, control_signal[joint_handles.index(joint_handle)])
        error = np.array(target) - get_positions(joint_handles)
    for joint_handle in joint_handles:
        sim.setJointTargetVelocity(joint_handle, 0)

In [165]:
def move_to_config(joint_handles: list[int], target: list[float], block=True):
    """
        Move to configuration using proportional control

        joint_handles: list of joint handles

        target: list of target angles

        block: whether to block until the target is reached

        """
    global cancel
    if 'move_to_config' in [t.name for t in threading.enumerate()]:
        print('Cancelling previous move to configuration')
        cancel.set()
        t = [t for t in threading.enumerate() if t.name == 'move_to_config'][0]
        t.join()
    cancel.clear()
    thread = threading.Thread(target=task, args=(joint_handles, target), name='move_to_config')
    print('Moving to configuration...')
    thread.start()
    if block:
        thread.join()

### Control Gripper

In [148]:
def close_gripper():
    global attached_shape
    index = 0
    while True:
        shape = sim.getObjects(index, sim.object_shape_type)

        if shape == -1:
            break
        elif(sim.getObjectInt32Param(shape,sim.shapeintparam_static)==0) and \
            (sim.getObjectInt32Param(shape,sim.shapeintparam_respondable)!=0) and \
            (sim.checkProximitySensor(objectSensor,shape)[0]==1):
            object_name = sim.getObjectName(shape)
            print('Object detected: ' + object_name)
            attached_shape = shape
            # Do the connection:
            sim.setObjectParent(attached_shape,connector, True)
            break
        index += 1
    sim.setJointTargetVelocity(finger_handles[0], -0.02)
    sim.setJointTargetVelocity(finger_handles[1], -0.02)
    while True:
        if sim.getJointForce(finger_handles[0]) > 9 and sim.getJointForce(finger_handles[1]) > 9:
            sim.setJointTargetVelocity(finger_handles[0], 0)
            sim.setJointTargetVelocity(finger_handles[1], 0)
            break

In [149]:
def open_gripper():
    global attached_shape
    if attached_shape != -1:
        sim.setObjectParent(attached_shape, -1, True)
    sim.setJointTargetVelocity(finger_handles[0], 0.02)
    sim.setJointTargetVelocity(finger_handles[1], 0.02)
    while True:
        if sim.getJointForce(finger_handles[0]) < -9 and sim.getJointForce(finger_handles[1]) < -9:
            sim.setJointTargetVelocity(finger_handles[0], 0)
            sim.setJointTargetVelocity(finger_handles[1], 0)
            break
    attached_shape = -1

In [150]:
open_gripper()

In [173]:
move_to_config(joint_handles, [0, 0, 0, 0, 0, 0], block=True)

Moving to configuration...


In [170]:
move_to_config(joint_handles, [.5*PI, .05*PI, .45*PI, 0, -.5*PI, PI/2])

Moving to configuration...


In [172]:
move_to_config(joint_handles, [.42*PI, .09*PI, .41*PI, 0, -.5*PI, PI/2], block=False)

Moving to configuration...


In [154]:
move_to_config(joint_handles, [.1*PI, .18*PI, .57*PI, .25*PI, 0.22, PI/2])

Moving to configuration...


In [155]:
move_to_config(joint_handles, [.33*PI, .18*PI, .57*PI, .25*PI, 0.22, PI/2])

Moving to configuration...


In [156]:
close_gripper()

In [157]:
move_to_config(joint_handles, [0, 0, 0, 0, 0, 0])

Moving to configuration...
